In [1]:
import pandas as pd
import mysql.connector
from mysql.connector.errors import Error

In [2]:
dados = pd.read_csv('dados/dados_pontos_selecionados.csv').drop_duplicates(subset='old_circuit_id', keep='first').dropna(subset=['new_circuit_id'])

In [ ]:
vdp = pd.read_csv('C:\\Users\\kmenezes\\OneDrive - Viasat, Inc\\Recorrente\\acesso_vdp\\dados\\dados_vdp_nov_21.csv')[['sai', 'idu_device_model', 'root_product_name', 'location', 'mac_address']]
vdp = pd.concat([vdp.drop('location',axis=1), vdp['location'].apply(lambda x: eval(str(x))).apply(pd.Series)], axis=1)
vdp['region'] = vdp['region'].fillna(vdp['sai'].apply(lambda x: x[:2]))
vdp.drop(columns=['country', 'country_code', 'postal_code'], inplace=True)
vdp.columns = ['new_circuit_id', 'modem', 'plan', 'mac_address', 'city', 'latitude', 'longitude', 'state']
vdp['mac_address'] = vdp['mac_address'].str.replace(':','').str.upper()

In [ ]:
host = "45.224.130.174"
port = "9773"
user = "karl_menezes"
password = "vK2Tr09Psz"
database = "crm-producao"

connection = mysql.connector.connect(
    host= host,     
    database= database,  
    user= user,   
    password= password,
    port = port  
)

In [ ]:
lista = tuple(dados['new_circuit_id'])

sql_query = f''' SELECT
    accounts.circuit_id,
    accounts.email,
    accounts.phone_number_1
FROM 
    accounts
WHERE
    accounts.circuit_id IN {lista}
'''

df = pd.read_sql_query(sql_query, connection).drop_duplicates(subset='circuit_id', keep='last')


In [ ]:
dados = dados.merge(vdp, on='new_circuit_id', how='left')

In [ ]:
dados.to_csv('dados_pontos_selecionados.csv', index=False)

In [ ]:
grace = pd.read_csv('C:\\Users\\kmenezes\\OneDrive - Viasat, Inc\\Recorrente\\novo_grace\\sistema 3.0\\verificacao/adfasf.csv')

In [15]:
d1 = grace.sort_values('completed_date').drop_duplicates(subset='terminal_id', keep='last')[['terminal_id', 'completed_date']]
d1.columns = ['terminal_id', 'created_at']
d1['terminated_at'] = pd.to_datetime(d1['created_at']) + pd.DateOffset(months=60)

In [21]:
dados.merge(grace[['old_circuit_id', 'terminal_id']].merge(d1, on='terminal_id', how='left'), on = 'old_circuit_id', how='left').to_csv('dados/dados_pontos_selecionados.csv', index=False)

In [3]:
usage = pd.read_csv('dados/oct_hourly_use.csv')

In [ ]:
a1 = usage.groupby(['date']).agg({'Total':'sum'}).reset_index()
a1['Total'] = a1['Total'] / 1024 ** 4

fig2 = px.bar(a1, x='date', y='TBytes', title = 'Consumo de dados por dia do mês', color_discrete_sequence= [c1])

#fig.add_vline(x=37, line_width=3, line_dash="dash", line_color=c2)


fig2.update_xaxes(showgrid=True,  tickangle = 45)


fig2.update_layout(xaxis_title = 'Semana do Ano', yaxis_title = 'Consumo (TBytes)', legend_title = 'Detalhe')

0         2024-10-11 06:00:00
1         2024-10-11 06:00:00
2         2024-10-11 06:00:00
3         2024-10-10 04:00:00
4         2024-10-10 04:00:00
                  ...        
3262569   2024-07-07 02:00:00
3262570   2024-07-17 17:00:00
3262571   2024-11-11 11:00:00
3262572   2024-07-23 02:00:00
3262573   2024-09-15 11:00:00
Length: 3262574, dtype: datetime64[ns]

In [8]:
usage

,year,month,day,hour,mac_address,incomingBytes,outgoingBytes
0,2024,10,11,6,00A0BC556C90,68829985,2158510
1,2024,10,11,6,00A0BC33A38E,1801,968
2,2024,10,11,6,00A0BC459BBF,30047882,1773984
3,2024,10,10,4,00A0BC8E3AE0,95325038,9825723
4,2024,10,10,4,00A0BC9414D0,118009,716033
...,...,...,...,...,...,...,...
411930,2024,10,3,19,00A0BCA8BBC4,288,80
411931,2024,10,9,1,00A0BC9422F4,24793,1848816
411932,2024,10,8,13,00A0BC4EC344,328,0
411933,2024,10,11,20,00A0BCBE5F3C,0,0


In [33]:
usage[usage['month'] == 10].to_csv('dados/oct_hourly_use.csv', index=False)

In [34]:
apps = pd.read_csv('dados/apps_use.csv')

In [35]:
apps[apps['month'] == 10].to_csv('dados/oct_apps_use.csv', index=False)